# How educator compensation has changed over time

This analyis is based on this NCES report (http://nces.ed.gov/pubsearch/pubsinfo.asp?pubid=2016156) on instructional staff compensation from 1991 to 2011. This report presents nationwide figures, so I wanted to break out Connecticut's data for a  story on TrendCT.org using the following raw data:

* Nonfiscal data: https://nces.ed.gov/ccd/stnfis.asp
* Fiscal data: https://nces.ed.gov/ccd/stfis.asp
* Data documentation here: http://nces.ed.gov/ccd/pdf/stfis111agen.pdf

#### A note for nontechnical readers
I encourage nontechnical readers to look over my notebooks. There is a lot of code (in gray boxes), but sticking to just the headlines and copy text should still be decipherable for anyone trying to better understand my methodology, including the error checking I did and questions I set out to answer.

The reason there is so much code before I get to any conclusions is that the data is stored in a weirdly named zip file containing a spreadsheet for each year on the raw data web pages linked above. In order to download and expand all of those files without losing track of which file went with which year, I had to write a lot of code.

#### A note for technical readers
I tried and gave up on a different approach to importing this data, which you can see in the other notebooks in this directory. They might be useful, but I'm not making their documentation a priority.

# Importing data

It took a lot of work to import all these sheets, so this is a neater presentation than the in the notebooks I used to figure out how to do it. Skip down to the Analysis portion if you want to just see the calculations for the story

In [1]:
import pandas as pd, numpy as np, re

In [2]:
fiscal_links = pd.read_csv("data/2/fiscal_links.csv")
nonfiscal_links = pd.read_csv("data/2/nonfiscal_links.csv")

In [3]:
import urllib2, zipfile, os

In [4]:
def download(zip_url, path):
    response = urllib2.urlopen(zip_url)
    zipcontent= response.read()
    
    with open("tmp.zip", 'w') as f:
        f.write(zipcontent)
        f.close()
    

    zip_ref = zipfile.ZipFile("tmp.zip", 'r')
    zip_ref.extractall(path)
    zip_ref.close()

In [5]:
def import_excel_file(dir_path):
    #print "Looking for excel files in " + dir_path
    excel_files = []
    for filename in os.listdir(dir_path):
        if ".xls" in filename.lower():
            excel_files.append(dir_path + "/" + filename)
            
    if len(excel_files) == 0: 
        return None
    elif len(excel_files) == 1:
        ret_frame = pd.read_excel(excel_files[0])#[["SURVYEAR","STABR","E11","E12"]]
        #ret_frame["STABR"] = ret_frame["STABR"].apply(lambda x: str(x).upper().strip())
        ret_frame.columns = map(lambda x: str(x).strip().upper(), ret_frame.columns)
        return ret_frame
    else:
        throw("Error: Multiple excel files found in " + dir_path)

In [6]:
import re
def clean_fiscal_colname(col_name):
    return  re.sub("FISCAL_YEAR_","",re.sub(r"[^\w]+","_",re.sub(r"[\(\)]","",col_name).strip().upper()))

In [7]:
def get_frameset(folder_path, links_frame):
    frame_set = {}
    for row, val in links_frame.iterrows():
        try:
            year = val["School year"]
            zip_link = val["Excel Link_link"]
        except:
            print "Error"
            print val
        path =  folder_path + str(year)
        #print path
        #download(zip_link, path)
        frame_set[clean_fiscal_colname(year)] = import_excel_file(path)
        
    return frame_set

In [8]:
fiscal_frames = get_frameset("data/2/fiscal/", fiscal_links)
nonfiscal_frames = get_frameset("data/2/nonfiscal/", nonfiscal_links)


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [9]:
# Need this to translate state names to postal abbreviations for 
# the couple of years when postals weren't used
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [10]:
# This structure describes which columns contain the desired data in each file
# since a couple of them don't match
good_years = {
    "nonfiscal":{
        "2013":{
            "sheet_name":'2013_14_V_1A',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
         "2012":{
            "sheet_name":'2013_14_V_1A',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2011":{
            "sheet_name": '2011_12_V_1A',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2010":{
            "sheet_name": '2010_11_V_1A',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2009":{
            "sheet_name": '2009_10_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2008":{
            "sheet_name": '2008_09_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2007":{
            "sheet_name":  '2007_08_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2006":{
            "sheet_name":   '2006_07_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2005":{
            "sheet_name": '2005_06_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2004":{
            "sheet_name": '2004_05_V_1F',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2003":{
            "sheet_name": '2003_04_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2002":{
            "sheet_name": '2002_03_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2001":{
            "sheet_name":  '2001_02_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "2000":{
            "sheet_name":  '2000_01_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1999":{
            "sheet_name": '1999_2000_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1998":{
            "sheet_name": '1998_99_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1997":{
            "sheet_name": '1997_98_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1996":{
            "sheet_name": '1996_97_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1995":{
            "sheet_name": '1995_96_V_1B',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1994":{
            "sheet_name": '1994_95_V_1B',
            "state_column":"STATE",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1993":{
            "sheet_name": '1993_94_V_1B',
            "state_column":"STATE",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1992":{
            "sheet_name": '1992_93_V_1C',
            "state_column":"STABR",
            "teacher_column":"TOTTCH",
            "aides_column":"AIDES"
        },
        "1991":{
            "sheet_name": '1991_92_V_1C',
            "state_column":"STABR",
            "teacher_column":'TOTAL TEACHERS', 
            "aides_column":'INSTRUCTIONAL AIDES',
        }
    },
    "fiscal":{
        "2013":{
            "sheet_name": '2013_V_1A_PROVISIONAL',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",
        },
        "2012":{
            "sheet_name": '2012_V_1A_PROVISIONAL',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",
        },
        "2011":{
            "sheet_name":'2011_V_1A_PRELIMINARY',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2010":{
            "sheet_name":'2010_V_1A_PROVISIONAL',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2009":{
            "sheet_name":'2009_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2008":{
            "sheet_name":'2008_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2007":{
            "sheet_name":'2007_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2006":{
            "sheet_name":'2006_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2005":{
            "sheet_name":'2005_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2004":{
            "sheet_name":'2004_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2003":{
            "sheet_name":'2003_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2002":{
            "sheet_name":'2002_V_1D_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "2000":{
            "sheet_name":'2000_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        } ,
        "2001":{
            "sheet_name":'2001_V_1B_REVISED',#'2000_V_1A_PROVISIONAL',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1999":{
            "sheet_name":'1999_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1998":{
            "sheet_name":'1998_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1997":{
            "sheet_name":'1997_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1996":{
            "sheet_name":'1996_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1995":{
            "sheet_name":'1995_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1994":{
            "sheet_name":'1994_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1993":{
            "sheet_name":'1993_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1992":{
            "sheet_name":'1992_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        },
        "1991":{
            "sheet_name":'1991_V_1B_REVISED',
            "state_column":"STABR",
            "E11_column":"E11",
            "E12_column":"E12",        
        } 
    }
}

def get_year_nonfiscal(year):
    record = good_years["nonfiscal"][year]
    sheet_name = record["sheet_name"]
    state_column = record["state_column"]
    teacher_column = record["teacher_column"]
    aides_column = record["aides_column"]

    try:
        fr = nonfiscal_frames[sheet_name][[state_column,teacher_column,aides_column]]
        fr[state_column] = fr[state_column].apply(lambda x: str(x).upper().strip())
        fr.columns = "STATE","TEACHERS","AIDES"
    
    except:
        print nonfiscal_frames[sheet_name].columns
    
    #fr["YEAR"] = year
    return fr

def get_year_fiscal(year):
    record = good_years["fiscal"][year]
    sheet_name = record["sheet_name"]
    
    state_column = record["state_column"]
    E11_column = record["E11_column"]
    E12_column = record["E12_column"]

    fr = fiscal_frames[sheet_name][[state_column,E11_column,E12_column]]
    fr[state_column] = fr[state_column].apply(lambda x: str(x).upper().strip())
    fr.columns = "STATE","E11","E12"
    #fr["YEAR"] = year
    return fr

def postal(state_name):
    if state_name.title() in us_state_abbrev:
        return us_state_abbrev[state_name.title()]
    return state_name

def get_year(year):
    left = get_year_fiscal(year).set_index("STATE")
    right = get_year_nonfiscal(year)
    
    # Convert state names to postal abbreviations
    right["STATE"] = right["STATE"].apply(lambda x: postal(x))
    right = right.set_index("STATE")

    
    fr = left.join(right)
    fr["E11"] = pd.to_numeric(fr["E11"], errors="coerce")
    fr = fr[fr["E11"].notnull()]
    fr["year"] = year
    return fr

In [11]:
def get_state_year(state, year):
    fr = get_year(str(year))
    #fr = fr[fr["STATE"] == state]
    return fr.loc[state]

def get_state (state):
    record = {}
    total_dollars = 0
    total_educators = 0
    for year in range(1991,2014):
        record[year] = get_state_year(state, year)[["E11","E12","TEACHERS","AIDES"]]
        continue
        row = {
            "DOLLARS": fr["E11"],
            "EDUCATORS": fr["TEACHERS"]
        }
        
        row = {
            state: float(fr["E11"]) / float(fr["TEACHERS"])
        }
        
        try:
            #row = (float(fr["E11"]) + float(fr["E12"])) / (float(fr["TEACHERS"]) + float(fr["AIDES"]))
            row = (float(fr["E11"]) + float(fr["E12"]))
        except:
            row = None
            
        record[year] = row
        
    return record

In [12]:
all = {}
for state_name in us_state_abbrev.keys():
    stabr = us_state_abbrev[state_name]
    print "Getting:", stabr

    all[stabr] = get_state(stabr)

Getting: MS


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

Getting: OK
Getting: DE
Getting: MN
Getting: IL
Getting: AR
Getting: NM
Getting: IN
Getting: LA
Getting: TX
Getting: WI
Getting: KS
Getting: CT
Getting: CA
Getting: WV
Getting: GA
Getting: ND
Getting: PA
Getting: AK
Getting: MO
Getting: SD
Getting: CO
Getting: NJ
Getting: WA
Getting: NY
Getting: NV
Getting: MD
Getting: ID
Getting: WY
Getting: AZ
Getting: IA
Getting: MI
Getting: UT
Getting: VA
Getting: OR
Getting: MT
Getting: NH
Getting: MA
Getting: SC
Getting: VT
Getting: FL
Getting: HI
Getting: KY
Getting: RI
Getting: NE
Getting: OH
Getting: AL
Getting: NC
Getting: TN
Getting: ME


In [13]:
# Return all the postal abbreviations of US states as a list
def postals():
    ret = []
    for state_name in us_state_abbrev:
        ret.append(us_state_abbrev[state_name])
    return ret
all_postals = postals()

# Analysis

Here's the numer-crunching. It's still code-heavy, so I'll put clear subheds on the output I used in the story for charts and copy.

In [14]:
def sum_col(year, states, col):
    total = 0
    for state in states:
        try: 
            total += float(all[state][year][col])
        except:
            print "ERROR Converting to numeric value: " + all[state][year][col] + " in ", state , str(year), " " + col
            total
    return total

def sum_salaries(year, states):
    return sum_col(year, states, "E11")

def sum_benefits(year, states):
    return sum_col(year, states, "E12")

def compensation(year, states):
    return sum_salaries(year, states) + sum_benefits(year, states)


def sum_aides (year, states):
    return sum_col (year,states,"AIDES")
def sum_teachers (year, states):
    return sum_col(year, states,"TEACHERS")
def educators(year, states):
    return sum_teachers(year, states) + sum_aides(year, states)

def per_teacher(year, states):
    return compensation (year, states) / educators(year, states)

In [15]:
def all_years(f, state):
    ret = {}
    values = []
    indexes = []
    for year in range(1991, 2014):
        indexes.append(year)
        values.append(int(f(year, state)))
        ret[year] = f(year, state)
    return indexes, values

In [127]:
from uscpi import UsCpi

def inflate(dollars, from_year, to_year=2011):
    cpi = UsCpi()
    return int(cpi.value_with_inflation(dollars, from_year, to_year))

def inflater_2011(row):
    #print "inflating", row
    #print "---------"
    year = row["year"]
    old_dollars = row["amount"]
    #print "inflating", old_dollars, " in ", str(year)
    cpi = UsCpi()
    return int(cpi.value_with_inflation(old_dollars, year, 2011))

def pay_timeline(cols):
    pay_timeline = all_years(per_teacher, cols)
    fr = pd.DataFrame(pay_timeline[1],index=pay_timeline[0]).reset_index()
    fr.columns = "year","amount"
    fr["2011 dollars"] = fr.apply(inflater_2011, axis=1)
    return fr
    #return fr.to_csv(index=False,sep="\t")
    #return fr[["year","2011 dollars"]].to_csv(index=False,sep="\t")
#inflater_2011(1991, 100)

def csvframe(fr):
    return fr.to_csv(index=False,sep="\t")

# * Pay over time, adjusted for inflation

The following two tables show the compensation (salary and benefits) of instructional staff. The first one is CT, and the second is nationwide. It shows both the nominal dollar value in the "amount" column" and the 2011-adjusted dollars.

Note that for this and subsequent 2011-adjusted tables, the data in years after 2011 doesn't necessarily make sense

In [147]:
print csvframe(pay_timeline(["CT"]).set_index("year")\
.join(pay_timeline(postals()).set_index("year")
                                            ,rsuffix="_US",lsuffix="_CT")\
               .reset_index()[["year","2011 dollars_CT","2011 dollars_US"]])

ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES
year	2011 dollars_CT	2011 dollars_US
1991	85229	65691
1992	87325	65912
1993	86267	65923
1994	85898	65683
1995	83550	66264
1996	81927	65217
1997	77978	64954
1998	76769	64978
1999	76468	64834
2000	75994	66090
2001	75854	67082
2002	78152	69165
2003	79941	70331
2004	85464	69920
2005	87813	70287
2006	89609	70805
2007	89216	72315
2008	77384	72870
2009	86274	75385
2010	86559	77612
2011	86971	75547
2012	86386	73136
2013	86882	73171



In [133]:
pay_timeline(postals())

ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES


,year,amount,2011 dollars
0,1991,39776,65691
1,1992,41111,65912
2,1993,42349,65923
3,1994,43275,65683
4,1995,44895,66264
5,1996,45491,65217
6,1997,46347,64954
7,1998,47086,64978
8,1999,48019,64834
9,2000,50595,66090


# * Was there really a decline in 2007 to 2008 in CT?

The table in the previos section shows a nominal and real (inflation-adjusted) decline in compensation costs per educator from 2007 to 2008 which caught my eye. 


#### Exec summary
It turns out there was a 17% increase in the number of instructional staff members (up 9,163 from 53045 to 62208), all but 4 of which were teachers, not aides. Meanwhile, the costs grew 5.6% (245 million), split even with 123 million in benefits and 123 million salaries. But that means the smaller benefit line grew at a faster pace than salaries - up 10.7%, compared with salary costs which moved at 3.8%.

#### Checking my work

I wanted to make sure that wasn't an error. In the next chunk of code, I verify that my math and code is right in calculating these columns, and then derive the conclusions in the "Exec summary" subsection above.

#### Check 1: Check the math and code

I see a big increase in the number of teachers

In [25]:
def test_year(year):
    print "========================================"
    print "Testing: ", year
    print "========================================"
    print 
    print "-------------------------"
    print "+ Educatour count tests + "
    print "-------------------------"
    print "Total teachers  : ", sum_teachers(year, ["CT"])
    print "Total aides     : ", sum_aides(year, ["CT"])
    print "Teachers + aides: ", educators(year, ["CT"])
    print "Teachers + aides: ", sum_teachers(year, ["CT"]) + sum_aides(year, ["CT"])
    print
    print "---------------------------"
    print "+ Compensation cost tests +"
    print "---------------------------"
    print "Salary costs  :", sum_salaries(year, ["CT"])
    print "Benefits costs:", sum_benefits(year, ["CT"])
    print "Sal + benefits:", compensation(year,["CT"])
    print "Sal + benefits:", sum_salaries(year, ["CT"]) + sum_benefits(year, ["CT"])
    print
    print "--------------------------"
    print "+ Per-teacher cost tests +"
    print "--------------------------"
    print "Cost per teacher:", per_teacher(year, ["CT"])
    print "Cost per teacher:", (sum_salaries(year, ["CT"]) \
                                + sum_benefits(year, ["CT"])) \
    / (sum_teachers(year, ["CT"]) + sum_aides(year, ["CT"]))

In [26]:
test_year(2007)

Testing:  2007

-------------------------
+ Educatour count tests + 
-------------------------
Total teachers  :  39304.0
Total aides     :  13741.0
Teachers + aides:  53045.0
Teachers + aides:  53045.0

---------------------------
+ Compensation cost tests +
---------------------------
Salary costs  : 3221094679.0
Benefits costs: 1141204140.0
Sal + benefits: 4362298819.0
Sal + benefits: 4362298819.0

--------------------------
+ Per-teacher cost tests +
--------------------------
Cost per teacher: 82237.7004242
Cost per teacher: 82237.7004242


In [27]:
test_year(2008)

Testing:  2008

-------------------------
+ Educatour count tests + 
-------------------------
Total teachers  :  48463.0
Total aides     :  13745.0
Teachers + aides:  62208.0
Teachers + aides:  62208.0

---------------------------
+ Compensation cost tests +
---------------------------
Salary costs  : 3343802807.0
Benefits costs: 1263905104.0
Sal + benefits: 4607707911.0
Sal + benefits: 4607707911.0

--------------------------
+ Per-teacher cost tests +
--------------------------
Cost per teacher: 74069.3787133
Cost per teacher: 74069.3787133


#### Check 2: Check the imported data against the original spreadsheet files

I see that the code and math is working as expected. Now I'm going to manually check the original Excel files and make sure the numbers in the columns line up with the data that was imported. I'll write my findings out to serve as a log as I check the data.

* Fiscal 2007: 
    * Filename: stfis071b.xls
    * E11: 3221094679
    * E12: 1141204140
    * CORRECT
* Fiscal 2008:
    * Filename: stfis081b.xls
    * E11: 3343802807
    * E12: 1263905104
    * CORRECT
* Nonfiscal 2007:
    * Filename: st071b.xls
    * TOTTCH: 39304
    * AIDES: 13741
    * CORRECT
* Nonfiscal 2008
    * Filename: st081c.xls
    * TOTTCH: 48463
    * AIDES: 13745
    * CORRECT


#### It's not wrong. Summarize the data

I confirmed in the above steps that the correct data was imported and computed correctly. So, what makes for the compensation decline? The following code generates a summary of the data

In [42]:
# Check the # and % change from year to year for:
# teacher count, aide count, combined educator count, salary costs and benefit costs
import json

def summarize (v1, v2):
    return {
        "first": v1,
        "second": v2,
        "dif": (v2 - v1),
        "pct": (v2 - v1) * 100 / v1,
    }

def change(year1, year2):
    
    
    summary = {
        "TEACHER COUNT":summarize(sum_teachers(year1,["CT"]),sum_teachers(year2,["CT"])),
        "AIDE COUNT":summarize(sum_aides(year1,["CT"]),sum_aides(year2,["CT"])),
        "EDUCATOR COUNT":summarize(educators(year1,["CT"]),educators(year2,["CT"])),
        "SALARY COSTS":summarize(sum_salaries(year1,["CT"]),sum_salaries(year2,["CT"])),
        "BENFT COSTS":summarize(sum_benefits(year1,["CT"]),sum_benefits(year2,["CT"])),
        "TOTAL COSTS":summarize(compensation(year1,["CT"]),compensation(year2,["CT"]))
        }
    
    print "Plain-english summary of change from ", year1,"to",year2
    print "--------------------------------------------------"
    for k in summary.keys():
        print "* ", k + " went from ", summary[k]["first"], "to", summary[k]["second"],\
        ". That's a change of", summary[k]["dif"],"or",summary[k]["pct"],"percent."
# return "The number of teachers grew " \
#+ str(dif_teacher) + " ("+ str(pct_teacher) + "%) from " \
#+ str(teachers1) + " to " + str(teachers2)

    return summary

summary = change(2007, 2008)
#print json.dumps(summary, indent=2)

Plain-english summary of change from  2007 to 2008
--------------------------------------------------
*  BENFT COSTS went from  1141204140.0 to 1263905104.0 . That's a change of 122700964.0 or 10.7518856355 percent.
*  EDUCATOR COUNT went from  53045.0 to 62208.0 . That's a change of 9163.0 or 17.2740126308 percent.
*  TEACHER COUNT went from  39304.0 to 48463.0 . That's a change of 9159.0 or 23.3029717077 percent.
*  TOTAL COSTS went from  4362298819.0 to 4607707911.0 . That's a change of 245409092.0 or 5.62568274624 percent.
*  SALARY COSTS went from  3221094679.0 to 3343802807.0 . That's a change of 122708128.0 or 3.80951633617 percent.
*  AIDE COUNT went from  13741.0 to 13745.0 . That's a change of 4.0 or 0.0291099628848 percent.


So we can see above that the teacher count grew at a far faster pace (23 %) than the costs (6%). The Aide count barely moved, adding just 4 aides. 

Does that mean more new teachers at lower salaries? Maybe more part-time teachers made up that total FTE count. Can't say from the data.

# * Total spending on instructional staff over time

Here I make sure that the numbers in my data set support the bullet point in (http://nces.ed.gov/pubs2016/2016156.pdf) that says the only year spending on salary and benefit spending (aggregate, not per-teacher cost) declined was from 2010 to 2011. This isn't central to my story, but it would help identify any problems with their or my analysis if I couldn't replicate that conclusion. Indeed, I draw the same conclusion.

In [23]:
# helper function to make tuple pairs into a dataframe
def tupleframe(frame_tuple, columns=False):
    return pd.DataFrame(frame_tuple[1],index=frame_tuple[0]).reset_index()

fr = tupleframe(all_years(compensation, postals()))
fr.columns = "year", "amount"
fr["2011 $"] = fr.apply(inflater_2011, axis=1)
fr

,year,amount,2011 $
0,1991,112806928363,186304534941
1,1992,118394428336,189818419924
2,1993,124847148033,194345969767
3,1994,130637836016,198283024261
4,1995,138587727698,204552394230
5,1996,144571202440,207263873203
6,1997,152864617313,214238094416
7,1998,160698507380,221762954303
8,1999,169347010842,228647942807
9,2000,180967011605,236391048916


In [28]:
fr = tupleframe(all_years(compensation, ["CT"]))
fr.columns = "year", "amount"
fr["2011 $"] = fr.apply(inflater_2011, axis=1)
fr

,year,amount,2011 $
0,1991,2060646358,3403228569
1,1992,2221476305,3561629783
2,1993,2255736547,3511440298
3,1994,2384781232,3619637689
4,1995,2467542282,3642037358
5,1996,2517697877,3609486569
6,1997,2589442842,3629075909
7,1998,2710420451,3740363594
8,1999,2857356593,3857928779
9,2000,3025098500,3951583225


# Summarizing the change from 1991 to 2011, without inflation

Since I built that handy summary tool to check my work from 2007 to 2008, I'll apply it below to summarize the net change from 1991 to 2011, the span of time looked at in the NCES analysis

In [47]:
change(1991, 2011)

Plain-english summary of change from  1991 to 2011
--------------------------------------------------
*  BENFT COSTS went from  460789724.0 to 1448012365.0 . That's a change of 987222641.0 or 214.245802278 percent.
*  EDUCATOR COUNT went from  39930.0 to 58058.01 . That's a change of 18128.01 or 45.3994740796 percent.
*  TEACHER COUNT went from  34383.0 to 43804.81 . That's a change of 9421.81 or 27.4025245034 percent.
*  TOTAL COSTS went from  2060646358.0 to 5049417843.0 . That's a change of 2988771485.0 or 145.040485642 percent.
*  SALARY COSTS went from  1599856634.0 to 3601405478.0 . That's a change of 2001548844.0 or 125.108012897 percent.
*  AIDE COUNT went from  5547.0 to 14253.2 . That's a change of 8706.2 or 156.953308094 percent.


{'AIDE COUNT': {'dif': 8706.2,
  'first': 5547.0,
  'pct': 156.95330809446548,
  'second': 14253.2},
 'BENFT COSTS': {'dif': 987222641.0,
  'first': 460789724.0,
  'pct': 214.24580227835116,
  'second': 1448012365.0},
 'EDUCATOR COUNT': {'dif': 18128.009999999995,
  'first': 39930.0,
  'pct': 45.399474079639354,
  'second': 58058.009999999995},
 'SALARY COSTS': {'dif': 2001548844.0,
  'first': 1599856634.0,
  'pct': 125.10801289711063,
  'second': 3601405478.0},
 'TEACHER COUNT': {'dif': 9421.809999999998,
  'first': 34383.0,
  'pct': 27.402524503388296,
  'second': 43804.81},
 'TOTAL COSTS': {'dif': 2988771485.0,
  'first': 2060646358.0,
  'pct': 145.04048564164157,
  'second': 5049417843.0}}

# How costs have changed, with inflation

This table shows the total salary plus benefit compensation costs, nominal ("amount") and adjusted ("2011 $") over time. Values for years after 2011 aren't useful

In [53]:
# Adjust the "amount" column in a table with "year" and "amount" columns
def adjust_2011(frame):
    fr = pd.DataFrame(frame[1],
                      index=frame[0]).reset_index()

    fr.columns = "year","amount"
    fr["2011 $"] = fr.apply(inflater_2011, axis=1)
    return fr

adjust_2011(all_years(compensation,["CT"]))

,year,amount,2011 $
0,1991,2060646358,3403228569
1,1992,2221476305,3561629783
2,1993,2255736547,3511440298
3,1994,2384781232,3619637689
4,1995,2467542282,3642037358
5,1996,2517697877,3609486569
6,1997,2589442842,3629075909
7,1998,2710420451,3740363594
8,1999,2857356593,3857928779
9,2000,3025098500,3951583225


# How much did the cost of salaries and benefits change from 1991 to 2011 after accounting for inflation?

Here's the two pertinant years from the table above:

| Year | Nominal | 2011 $ |
|---|---|---|
| 1991| 2060646358 |	3403228569
| 2011 | 5049417843 | 5049417843 | 

From the below computation we see that that's a 48% increase.

In [49]:
summarize(3403228569, 5049417843)

{'dif': 1646189274, 'first': 3403228569, 'pct': 48, 'second': 5049417843}

In [83]:
# The same calculation utilizing functions made for previos work
summarize(inflate(compensation(1991,["CT"]),1991),
         compensation(2011, ["CT"]))

{'dif': 1646189274.0,
 'first': 3403228569,
 'pct': 48.37139911774172,
 'second': 5049417843.0}

#### Break out the salaries change over time

From the below computation we see salaries went from 2642218439 to 3601405478 (in 2011 dollars), a 36% increase

In [76]:
summarize(inflate(sum_salaries(1991,["CT"]),1991),
         sum_salaries(2011, ["CT"]))

{'dif': 959187039.0,
 'first': 2642218439,
 'pct': 36.3023368863864,
 'second': 3601405478.0}

#### Break out benefits change over time

In [82]:
summarize(inflate(sum_benefits(1991,["CT"]),1991),
         sum_benefits(2011, ["CT"]))

{'dif': 687002235.0,
 'first': 761010130,
 'pct': 90.27504469618559,
 'second': 1448012365.0}

#### Break out the change in teachers over time

In [86]:
summarize(sum_teachers(1991,["CT"]),
         sum_teachers(2011, ["CT"]))

{'dif': 9421.809999999998,
 'first': 34383.0,
 'pct': 27.402524503388296,
 'second': 43804.81}

#### Break out the per educator cost change over time


In [99]:
summarize(inflate(per_teacher(1991,["CT"]),1991),
         per_teacher(2011, ["CT"]))

{'dif': 1742.941391032873,
 'first': 85229,
 'pct': 2.0450097866135626,
 'second': 86971.94139103287}

#### Break out the change in aides over time


In [88]:
summarize(sum_aides(1991,["CT"]),
         sum_aides(2011, ["CT"]))

{'dif': 8706.2, 'first': 5547.0, 'pct': 156.95330809446548, 'second': 14253.2}

In [89]:
summarize(educators(1991,["CT"]),
         educators(2011, ["CT"]))

{'dif': 18128.009999999995,
 'first': 39930.0,
 'pct': 45.399474079639354,
 'second': 58058.009999999995}

#### Proportion of aides to all educators in 1991 and 2011


In [116]:
sum_aides(1991,["CT"]) * 100 / educators(1991,["CT"])

13.891810668670173

In [117]:
sum_aides(2011,["CT"]) * 100 / educators(2011,["CT"])

24.549928597277106

#### The number of aides has increased far faster than the number of teachers

This is intersting. If aides make up a larger share of the workforce, one would expect that to bring down the cost per instructional staff member, assuming aides are paid less than teachers. This could account for why the cost per employee has basically treaded water with inflation.

We can see from the computation below that the number of aides did grow far faster than teachers nationwide, but not nearly as fast as the rate in Connecticut.

In [105]:
print "Change in teacher count nationwide"
print summarize(sum_teachers(1991,postals()),
         sum_teachers(2011, postals()))
print
print "Change in aide count nationwide"
print summarize(sum_aides(1991,postals()),
         sum_aides(2011, postals()))
print
print "Change in combined educator count nationwide"
print summarize(educators(1991,postals()),
         educators(2011, postals()))

Change in teacher count nationwide
{'second': 3096984.4900000007, 'dif': 671087.4900000007, 'pct': 27.66347829277173, 'first': 2425897.0}

Change in aide count nationwide
ERROR Converting to numeric value: M in  NV 1991  AIDES
{'second': 708931.5899999999, 'dif': 298817.58999999985, 'pct': 72.86207981195469, 'first': 410114.0}

Change in combined educator count nationwide
ERROR Converting to numeric value: M in  NV 1991  AIDES
{'second': 3805916.0800000005, 'dif': 969905.0800000005, 'pct': 34.19962334419721, 'first': 2836011.0}


#### What's the proportion of aides to all educators nationwide

In [115]:
sum_aides(1991,postals()) * 100 / educators(1991,postals())

ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES


14.460945320733947

In [114]:
sum_aides(2011,postals()) * 100 / educators(2011,postals())

18.62709463630632

In [91]:
tupleframe(all_years(sum_aides, ["CT"]))

,index,0
0,1991,5547
1,1992,6592
2,1993,6178
3,1994,6822
4,1995,7520
5,1996,7506
6,1997,8881
7,1998,9950
8,1999,10544
9,2000,10954


# Now let's do the same thing for 1993 to 2013

Since more recent data is avaialable

In [103]:
print "Salary cost change"
print summarize(inflate(sum_salaries(1993,["CT"]),1993, 2013),
         sum_salaries(2013, ["CT"]))
print
print "Benefits cost change"
print summarize(inflate(sum_benefits(1993,["CT"]),1993, 2013),
         sum_benefits(2013, ["CT"]))
print
print "Combined cost change"
print summarize(inflate(compensation(1993,["CT"]),1993, 2013),
         compensation(2013, ["CT"]))
print
print "Per instructor cost change"
print summarize(inflate(per_teacher(1993,["CT"]),1993, 2013),
         per_teacher(2013, ["CT"]))



Salary cost change
{'second': 3671907651.0, 'dif': 693296125.0, 'pct': 23.275815558634886, 'first': 2978611526}

Benefits cost change
{'second': 1633961459.0, 'dif': 975966628.0, 'pct': 148.32436092495686, 'first': 657994831}

Combined cost change
{'second': 5305869110.0, 'dif': 1669262752.0, 'pct': 45.90166181522141, 'first': 3636606358}

Per instructor cost change
{'second': 89979.19896187288, 'dif': 637.198961872884, 'pct': 0.7132132276789013, 'first': 89342}


# Timeline of salaries and benefits


In [204]:
def inflate_timeline_2011(f, states):
    df = tupleframe(all_years(f, states))
    df.columns = "year", "amount"
    df["2011 $"] = df.apply(inflater_2011, axis=1)
    df = df.drop("amount", axis=1)
    return df
    
def compensation_timeline(states):
    total = inflate_timeline_2011(compensation, states)
    benefits = inflate_timeline_2011(sum_benefits,states)
    salaries = inflate_timeline_2011(sum_salaries,states)

    total.columns="year","total"
    benefits.columns="year","benefits"
    salaries.columns="year","salaries"

    total = total.set_index("year")
    benefits = benefits.set_index("year")
    salaries = salaries.set_index("year")


    compensation_timeline = benefits\
    .join(salaries)\
    .join(total)

    compensation_timeline["ben_pct"] = compensation_timeline["benefits"] \
    * 100 / compensation_timeline["total"]
    compensation_timeline["sal_pct"] = compensation_timeline["salaries"] \
    * 100 / compensation_timeline["total"]
    
    return compensation_timeline

ct_compensation_timeline = compensation_timeline(["CT"])
us_compensation_timeline = compensation_timeline(postals())

#ct_salaries = tupleframe(all_years(compensation, ["CT"]))
#ct_salaries.columns = "year", "amount"
#ct_salaries["2011 $"] = fr.apply(inflater_2011, axis=1)
#ct_salaries = ct_salaries.drop("amount", axis=1)
#ct_salaries

In [208]:
# This will do for charting the percentage of benefits relative to the overall
# compensation costs over time
print csvframe(ct_compensation_timeline[["ben_pct"]]\
.join(us_compensation_timeline[["ben_pct"]],rsuffix="_US",lsuffix="_CT").reset_index())

year	ben_pct_CT	ben_pct_US
1991	22.3614169478	19.7715844854
1992	18.0832318416	19.9505942896
1993	18.0936501287	20.8281138428
1994	18.2522981515	20.9608909295
1995	18.7697498077	21.0152134884
1996	18.285998476	20.904632785
1997	18.5746773257	20.8765489825
1998	19.0712905864	20.2909175261
1999	19.4292924245	20.0752775505
2000	19.9480655251	19.9782964544
2001	20.4778801982	20.3861474278
2002	20.895191838	20.5260465915
2003	21.0872486943	21.3890603136
2004	22.0666464843	22.4573606591
2005	24.9307288235	23.5161694668
2006	26.2947047593	24.0243290532
2007	26.160613639	24.5967081256
2008	27.4302349017	25.1241440899
2009	27.8557091331	25.0316989211
2010	27.8977446171	25.6100635631
2011	28.676818002	26.1299992415
2012	30.8137131238	26.7881310423
2013	30.7953593443	27.2335475174



# Timeline of share of aides

The final table needed to make the last chart: The percentage of aides relative to all teaching staff over time.

In [226]:
def teachers_timeline(states):
    teachers = tupleframe(all_years(sum_teachers, states))
    aides = tupleframe(all_years(sum_aides, states))
    total = tupleframe(all_years(educators, states))
    
    teachers.columns="year","teachers"
    aides.columns="year","aides"
    total.columns="year","total"
    
    teachers = teachers.set_index("year")
    aides = aides.set_index("year")
    total = total.set_index("year")
    
    timeline = teachers.join(aides).join(total)
    timeline ["pct_aides"] = timeline["aides"] * 100 / timeline["total"]
    return timeline.reset_index()
    
ct_teachers_timeline = teachers_timeline(["CT"])
us_teachers_timeline = teachers_timeline(postals())


ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES
ERROR Converting to numeric value: M in  NV 1991  AIDES


In [234]:
print csvframe(ct_teachers_timeline[["year","pct_aides"]].set_index("year")\
.join(us_teachers_timeline[["year","pct_aides"]].set_index("year"),
     rsuffix="_US",lsuffix="_CT").reset_index())

year	pct_aides_CT	pct_aides_US
1991	13.8918106687	14.4609453207
1992	16.1628049528	14.8250224753
1993	15.1778694969	15.2697864788
1994	16.1896625374	15.6683519094
1995	17.2516632255	16.001998136
1996	17.0370202238	16.2321884161
1997	19.0829197018	16.8710249284
1998	20.4219859612	17.2210513115
1999	20.8994866306	17.6035562522
2000	21.0661948536	17.9000230377
2001	22.1088942756	18.355186531
2002	22.2099610093	17.9352740919
2003	21.445389992	18.3493724551
2004	24.640270307	18.6241680212
2005	23.9348346909	18.0791035684
2006	25.4796243022	18.3069078429
2007	25.9044207748	18.3217332637
2008	22.0952289095	18.5527564753
2009	24.8759219687	18.7593410511
2010	26.6885145169	19.0955626891
2011	24.5495883427	18.6270795257
2012	26.3265894483	19.1581845467
2013	26.3265894483	19.1581845467

